In [14]:
import autogen
from autogen.agentchat.groupchat import GroupChat,GroupChatManager
from autogen.coding import LocalCommandLineCodeExecutor
from autogen.agentchat.user_proxy_agent import UserProxyAgent
from autogen.agentchat.assistant_agent import AssistantAgent

In [15]:
config_list = [ 
    {
        "model": "llama3",
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama",
        "price": [0.01, 0.02],
    }
]

In [16]:
llm_config={
    "cache_seed":42,
    "temperature":0,
    "config_list":config_list,
    "timeout":120,
}

In [17]:
user_proxy=autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved "
                   "by this admin.",
    code_execution_config={
        "work_dir": "code",
        "use_docker": False
    },   
    human_input_mode="TERMINATE",       
)

In [18]:
code_executer = autogen.UserProxyAgent(
    name="code_executor",
    system_message="Executor. Execute the code written by the engineer and report the result. Reply 'TERMINATE' if the task is done",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "executor": LocalCommandLineCodeExecutor(work_dir="coding"),
    },
)

In [19]:
planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer.
""",
    llm_config=llm_config,
)

In [20]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message="""Engineer.You are working as a Senior Frontend Engineer. Write your only thing in HTML code.
""",
)

In [21]:
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the "
                   "plan includes adding verifiable info such as source URL.",
    llm_config=llm_config,
)

In [22]:
def _reset_agents():
    planner.reset()
    engineer.reset()
    critic.reset()
    code_executer.reset()

In [23]:
_reset_agents()

In [24]:
groupchat = GroupChat(agents=[user_proxy, planner,engineer, critic], messages=[], max_round=12, speaker_selection_method="round_robin")
manager = GroupChatManager(groupchat=groupchat, llm_config={"config_list": config_list})

In [25]:
user_proxy.initiate_chat(
    manager,
    message="""
I'm getting an error. "Buttons do not have an accessible name" error. My first button error is '<<button class="carousel-control-prev" type="button" data-bs-target="#carouselExample" data-bs- slide="Previous slider" alt="Previous slider">>' My next error is <<button class="carousel-control-next" type="button" data-bs-target="#carouselExample" data-bs-slide="Next slider" alt="Next slider">> I need an engineer to solve these errors. The description of my error simply states that a button does not have an accessible name, which is necessary for screen readers and other accessibility tools to recognize it. This may prevent visually impaired users from understanding the purpose of the buttons when using web pages or applications.
""",
)

Admin (to chat_manager):


I'm getting an error. "Buttons do not have an accessible name" error. My first button error is '<<button class="carousel-control-prev" type="button" data-bs-target="#carouselExample" data-bs- slide="Previous slider" alt="Previous slider">>' My next error is <<button class="carousel-control-next" type="button" data-bs-target="#carouselExample" data-bs-slide="Next slider" alt="Next slider">> I need an engineer to solve these errors. The description of my error simply states that a button does not have an accessible name, which is necessary for screen readers and other accessibility tools to recognize it. This may prevent visually impaired users from understanding the purpose of the buttons when using web pages or applications.


--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

I'd be happy to help you with that!

Here's a plan to resolve the "Buttons do not have an accessible na

ChatResult(chat_id=None, chat_history=[{'content': '\nI\'m getting an error. "Buttons do not have an accessible name" error. My first button error is \'<<button class="carousel-control-prev" type="button" data-bs-target="#carouselExample" data-bs- slide="Previous slider" alt="Previous slider">>\' My next error is <<button class="carousel-control-next" type="button" data-bs-target="#carouselExample" data-bs-slide="Next slider" alt="Next slider">> I need an engineer to solve these errors. The description of my error simply states that a button does not have an accessible name, which is necessary for screen readers and other accessibility tools to recognize it. This may prevent visually impaired users from understanding the purpose of the buttons when using web pages or applications.\n', 'role': 'assistant'}, {'content': 'I\'d be happy to help you with that!\n\nHere\'s a plan to resolve the "Buttons do not have an accessible name" error:\n\n**Step 1: Engineer\'s Task**\n\n* The engineer w

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="""
"Heading elements are not in descending order" errors occur when HTML heading tags (for example, <h1>, <h2>, <h3>, etc.) are not used in the correct current order. This can lead to accessible issues and make it difficult for screen readers and other assistive technologies to accurately navigate and interpret the presentation of content. Can you solve this?
""",
)